# Install & import libraries

In [1]:
%pip install -q chess stockfish python-telegram-bot==13 cairosvg zstandard

Note: you may need to restart the kernel to use updated packages.


# Introduction to libraries used (with examples)

## Python Chess

python-chess is a chess library for Python, with move generation, move validation, and support for common formats. Read the documentation here: https://python-chess.readthedocs.io/en/latest/

In [ ]:
import chess, chess.svg
FEN = "r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - - 0 24"
Moves = "f2g3 e6e7 b2b1 b3c1 b1c1 h6c1"
Moves = Moves.split(" ")

In [ ]:
# Load FEN position
board = chess.Board(FEN)
# Display board
display(board)

In [ ]:
# Make moves
move_uci = Moves[0]
move_san = board.san(chess.Move.from_uci(move_uci))

# Make the move
board.push(chess.Move.from_uci(move_uci))

# Display
print("Move in Universal Chess Interface (UCI): {}".format(move_uci))
print("Move in Standard Algebraic Notation (SAN): {}".format(move_san))
display(board)

### Exporting SVG image and convert it to PNG format

In [ ]:
from cairosvg import svg2png
import chess.svg

boardsvg = chess.svg.board(board=board)
svg2png(bytestring=boardsvg,write_to='output.png')

## Stockfish

Stockfish is a chess engine used for evaluation positions and generating best moves. Read the python stockfish documentation here: https://pypi.org/project/stockfish/

In [4]:
import requests
from stockfish import Stockfish

# Download stockfish from github
stockfishURL = "https://github.com/official-stockfish/Stockfish/releases/download/sf_16/stockfish-ubuntu-x86-64-avx2.tar"
response = requests.get(stockfishURL)

with open("stockfish.tar", "wb") as f:
    f.write(response.content)

# Uncompress the tar file
!tar -xvf stockfish.tar

stockfish/
stockfish/CITATION.cff
stockfish/stockfish-ubuntu-x86-64-avx2
stockfish/src/
stockfish/src/benchmark.cpp
stockfish/src/uci.cpp
stockfish/src/types.h
stockfish/src/position.cpp
stockfish/src/position.h
stockfish/src/tune.cpp
stockfish/src/benchmark.h
stockfish/src/search.cpp
stockfish/src/endgame.cpp
stockfish/src/thread.h
stockfish/src/nnue/
stockfish/src/nnue/evaluate_nnue.h
stockfish/src/nnue/nnue_common.h
stockfish/src/nnue/layers/
stockfish/src/nnue/layers/clipped_relu.h
stockfish/src/nnue/layers/simd.h
stockfish/src/nnue/layers/affine_transform_sparse_input.h
stockfish/src/nnue/layers/affine_transform.h
stockfish/src/nnue/layers/sqr_clipped_relu.h
stockfish/src/nnue/features/
stockfish/src/nnue/features/half_ka_v2_hm.h
stockfish/src/nnue/features/half_ka_v2_hm.cpp
stockfish/src/nnue/nnue_architecture.h
stockfish/src/nnue/evaluate_nnue.cpp
stockfish/src/nnue/nnue_feature_transformer.h
stockfish/src/nnue/nnue_accumulator.h
stockfish/src/incbin/
stockfish/src/incbin/incbin

In [ ]:
# Initialize stockfish from the downloaded path
stockfish = Stockfish(path="/content/stockfish/stockfish-ubuntu-x86-64-avx2")

# Set stockfish configurations
stockfish.set_depth(15)
stockfish.set_elo_rating(2000)

In [ ]:
FEN = "r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - - 0 24"
# Set to starting position
stockfish.set_fen_position(FEN)

# Instruct stockfish to generate k top moves
k = 10
top_moves = stockfish.get_top_moves(k)
print(top_moves)

[{'Move': 'b2b1', 'Centipawn': -441, 'Mate': None}, {'Move': 'f2e3', 'Centipawn': -208, 'Mate': None}, {'Move': 'e7f7', 'Centipawn': -17, 'Mate': None}, {'Move': 'e7c7', 'Centipawn': 27, 'Mate': None}, {'Move': 'e7d7', 'Centipawn': 121, 'Mate': None}, {'Move': 'f2g3', 'Centipawn': 606, 'Mate': None}, {'Move': 'f2c5', 'Centipawn': 659, 'Mate': None}, {'Move': 'a8g8', 'Centipawn': 943, 'Mate': None}, {'Move': 'b2c1', 'Centipawn': None, 'Mate': 8}, {'Move': 'a7a5', 'Centipawn': None, 'Mate': 8}]


In [ ]:
best_move = top_moves[0]["Move"]
print(best_move)

b2b1


## Lichess puzzle dataset

Lichess provides an open database with chess puzzles here: https://database.lichess.org/?ref=propelauth.com#puzzles . For the telegram bot, we download and filter the puzzles we want.

In [5]:
import requests, zstandard
import pandas as pd

# Download zst file
puzzleURL = "https://database.lichess.org/lichess_db_puzzle.csv.zst"
response = requests.get(puzzleURL)

with open("lichess_db_puzzle.csv.zst", "wb") as f:
    f.write(response.content)


# Uncompress zst file
with open("lichess_db_puzzle.csv.zst", "rb") as f:
    decomp = zstandard.ZstdDecompressor()
    with open("lichess_db_puzzle.csv", 'wb') as destination:
        decomp.copy_stream(f, destination)

In [6]:
df = pd.read_csv("lichess_db_puzzle.csv")
print(df.shape)
df.head()

(3466049, 10)


,PuzzleId,FEN,Moves,Rating,RatingDeviation,Popularity,NbPlays,Themes,GameUrl,OpeningTags
0,00008,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - ...,f2g3 e6e7 b2b1 b3c1 b1c1 h6c1,1758,75,94,4412,crushing hangingPiece long middlegame,https://lichess.org/787zsVup/black#48,NaN
1,0000D,5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - ...,d3d6 f8d8 d6d8 f6d8,1511,74,96,23054,advantage endgame short,https://lichess.org/F8M8OS71#53,NaN
2,0008Q,8/4R3/1p2P3/p4r2/P6p/1P3Pk1/4K3/8 w - - 1 64,e7f7 f5e5 e2f1 e5e6,1292,75,100,116,advantage endgame rookEndgame short,https://lichess.org/MQSyb3KW#127,NaN
3,0009B,r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP...,b6c5 e2g4 h3g4 d1g4,1088,74,86,547,advantage middlegame short,https://lichess.org/4MWQCxQ6/black#32,Kings_Pawn_Game Kings_Pawn_Game_Leonardis_Vari...
4,000Vc,8/8/4k1p1/2KpP2p/5PP1/8/8/8 w - - 0 53,g4h5 g6h5 f4f5 e6e5 f5f6 e5f6,1556,81,89,81,crushing endgame long pawnEndgame,https://lichess.org/l6AejDMO#105,NaN


File size is too large for PythonAnywhere server, remove rows/columns to reduce < 100 mb

In [7]:
df2 = df[df["Themes"].str.contains("short")]
df2 = df2[df2["Rating"] > 1350]

# Grab the first 1 million rows
df2 = df2.iloc[:1000000]

# Keep only FEN, moves and rating rows
df2 = df2[["FEN","Moves","Rating"]]
df2.head()

# Write cleaned csv file
df2.to_csv("chess_puzzles.csv")

In [50]:
import pandas as pd
rating = 1500
rating_upper = rating+50
rating_lower = rating-50
filtered_df = df[(df.Rating>=rating_lower) & (df.Rating<=rating_upper)]
row = df.sample()
df.max()["Rating"]
df.min()["Rating"]

1351

# Telegram Bot

For the telegram API, we use the python-telegram-bot **v13 (not v20)** library. Read the documentation here: https://github.com/python-telegram-bot/python-telegram-bot/blob/v13.x . Run the cells below and the bot should work (@NTU_chess_test_bot)

## Chess & Utility functions

## Telegram functions

### Misc

In [ ]:
import random
stockfish = Stockfish(path="/content/stockfish/stockfish-ubuntu-x86-64-avx2")
# Set stockfish configurations
stockfish.set_depth(10)
stockfish.set_elo_rating(2000)
board = chess.Board("r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - - 0 24")
# Instruct stockfish to generate k top moves
solution_uci = "b2b1"
solution_san = uci_to_san(board, solution_uci)


FEN = board.board_fen()
stockfish.set_fen_position(FEN)
quiz_answers_count = 4
top_moves = stockfish.get_top_moves(7)
answers = [uci_to_san(board, top_moves[i]["Move"]) for i, _ in enumerate(top_moves)]
answers = random.sample(answers, quiz_answers_count)
if solution_san in answers:
  answers.remove(solution_san)
else:
  answers.pop()
solution_ind = random.randint(0,quiz_answers_count)
answers.insert(solution_ind, solution_san)
display(board)
print(answers, solution_san)

In [ ]:
def generate_inline_chessboard(board):
    files = ["a", "b", "c", "d", "e", "f", "g", "h"]
    ranks = [str(i) for i in range(8,0,-1)]
    indices = ['♚','♛','♜','♝','♞','♟','⭘','♙','♘','♗','♖','♕','♔']
    new_indices = ['♚','♛','♜','♝','♞','♟',' ','♙','♘','♗','♖','♕','♔']
    new_indices = ['♔','♕','♖','♗','♘','♟',' ','♙','♞','♝','♜','♛','♚']

    unicode_board = [
        [new_indices[indices.index(c)] for c in row.split()]
        for row in board.unicode(invert_color=False).split('\n')
    ]


    inline_chessboard = [
        [InlineKeyboardButton(unicode_board[i][j], callback_data=f"{f}{r}") for j,f in enumerate(files)] for i,r in enumerate(ranks)
    ]

    if board.turn == chess.BLACK:
        for row in inline_chessboard:
          row.reverse()
        inline_chessboard.reverse()

    reply_markup = InlineKeyboardMarkup(inline_chessboard)
    return reply_markup
def button(update: Update, context: CallbackContext) -> None:
    """Parses the CallbackQuery and updates the message text."""
    query = update.callback_query

    # CallbackQueries need to be answered, even if no notification to the user is needed
    # Some clients may have trouble otherwise. See https://core.telegram.org/bots/api#callbackquery
    query.answer()

    context.bot.send_message(chat_id=update.effective_chat.id, text=f"{query.data}")

    dp.add_handler(CallbackQueryHandler(button))
def check_answers(update: Update, context: CallbackContext):
    text = update.message.text.strip()
    chat_id = update.effective_chat.id
    solutions = context.bot_data[chat_id]["solutions"]
    for solution in solutions.values():
        if text in solution:
            context.bot.send_message(chat_id=update.effective_chat.id, text=f"Good job! The solution was {solutions[1]}.")
            puzzle(update, context)
            break

    dp.add_handler(MessageHandler(Filters.text & (~Filters.command), check_answers))